In [1]:
# imports
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[1])

# Add to sys.path
sys.path.insert(0, parent_dir)

In [2]:
import torch 
import math
from torch import nn
import torch.nn.functional as F

from src.transformers.models.functionals import create_cross_attention_mask

In [3]:
embed_dim = 128
num_heads = 8
hidden_dim = 200
max_len = 5000
batch_size = 32
d_ff = hidden_dim
num_head = 8


In [4]:
src = torch.tensor(
    [[ 1,3,4,2,0,0,0],
    [ 1,3,4,4,2,0,0],
    [ 1,2,0,0,0,0,0]]
) # [ 3, 7]
    
# en_tensor = torch.randint(1, 10**4,[batch_size, seq_len], dtype=int)
tgt = torch.tensor(
    [[ 1,3,2,0,0,0,0,0],
    [ 1,3,3,3,2,0,0,0],
    [ 1,3,2,0,0,0,0,0]]
) # [ 3, 8]

print(src.shape, tgt.shape)

embedding = nn.Embedding(10**4, embedding_dim=embed_dim)
src_embedding = embedding(src)
tgt_embedding = embedding(tgt)

print(src_embedding.shape, tgt_embedding.shape)
cross_padding_mask = create_cross_attention_mask(tgt, src)


torch.Size([3, 7]) torch.Size([3, 8])
torch.Size([3, 7, 128]) torch.Size([3, 8, 128])


# **Layers**

In [5]:
from src.transformers.models.layers import EncoderLayer, DecoderLayer

encoder_layer = EncoderLayer(
    embed_dim=embed_dim,
    num_heads=num_head,
)    

decoder_layer = DecoderLayer(
    embed_dim=embed_dim,
    num_heads=num_head,
)  

In [6]:
memory = encoder_layer(
    src_embedding,
    self_is_causal = True
)

logits = decoder_layer(
    tgt_embedding,
    memory,
    self_is_causal = True,
    cross_padding_mask = cross_padding_mask
)

print(
    f"\n{src_embedding.shape=}",
    f"\n{memory.shape=}",
    f"\n{logits.shape=}"
)



src_embedding.shape=torch.Size([3, 7, 128]) 
memory.shape=torch.Size([3, 7, 128]) 
logits.shape=torch.Size([3, 8, 128])


# **Encoder**

In [7]:
from src.transformers.models.encoders import Encoder

encoder = Encoder(
    vocab_size = 10**4,
    embed_dim=embed_dim,
    num_heads=num_head,
    n_layers=2,
    max_len=5000,
)

print(encoder)

c:\Users\francesco.paletta\AppData\Local\anaconda3\envs\optimus\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Encoder(
  (embedding): Embedding(10000, 128)
  (positional_encoding): PositionalEncoding()
  (layers): ModuleList(
    (0-1): 2 x EncoderLayer(
      (self_attention): MultiHeadAttention(
        (w_q): Linear(in_features=128, out_features=128, bias=True)
        (w_k): Linear(in_features=128, out_features=128, bias=True)
        (w_v): Linear(in_features=128, out_features=128, bias=True)
        (w_o): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): FeedForward(
        (linear1): Linear(in_features=128, out_features=128, bias=True)
        (linear2): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (relu): ReLU()
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout)

In [8]:
memory = encoder(
    src,
    src_padding_mask = None,
    self_is_causal = True
)

print(
    f"\n{memory.shape=}",
)



memory.shape=torch.Size([3, 7, 128])


# **Decoders**

In [9]:
from src.transformers.models.decoders import Decoder

decoder = Decoder(
    vocab_size = 10**4,
    embed_dim=embed_dim,
    num_heads=num_head,
    n_layers=2,
    max_len=5000,
)

print(decoder)

Decoder(
  (embedding): Embedding(10000, 128)
  (positional_encoding): PositionalEncoding()
  (layers): ModuleList(
    (0-1): 2 x DecoderLayer(
      (self_attention): MultiHeadAttention(
        (w_q): Linear(in_features=128, out_features=128, bias=True)
        (w_k): Linear(in_features=128, out_features=128, bias=True)
        (w_v): Linear(in_features=128, out_features=128, bias=True)
        (w_o): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (cross_attention): MultiHeadAttention(
        (w_q): Linear(in_features=128, out_features=128, bias=True)
        (w_k): Linear(in_features=128, out_features=128, bias=True)
        (w_v): Linear(in_features=128, out_features=128, bias=True)
        (w_o): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): FeedForward(
        (linear1): Linear(in_features=128, out_features=128, bias=True)
 

In [10]:
logits = decoder(
    tgt,
    memory,
    self_is_causal=True,
    cross_padding_mask=cross_padding_mask
)

print(
    f"\n{logits.shape=}"
)



logits.shape=torch.Size([3, 8, 128])


# **Model**

In [11]:
from src.transformers.models.optimus_model import OptimusTransformer

config = {
    'src_vocab_size': 10**4,
    'tgt_vocab_size': 11**4,
    'embed_dim': 128,
    'num_heads': 16,
    'n_layers': 1,
    'hidden_dim': 256,
    'max_seq_length': 100,
    'dropout': 0.1,
    'batch_size': 32,
    'num_epochs': 10,
    'warmup_steps': 4000,
    'label_smoothing': 0.1
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = OptimusTransformer(
src_vocab_size=config['src_vocab_size'],
tgt_vocab_size=config['tgt_vocab_size'],
n_layers=config['n_layers'],
embed_dim=config['embed_dim'],
num_heads=config['num_heads'],
max_len=config['max_seq_length'],
dropout=config['dropout'],
).to(device)

print(model)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


src_vocab_size=10000, tgt_vocab_size=14641


OptimusTransformer(
  (encoder): Encoder(
    (embedding): Embedding(10000, 128)
    (positional_encoding): PositionalEncoding()
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attention): MultiHeadAttention(
          (w_q): Linear(in_features=128, out_features=128, bias=True)
          (w_k): Linear(in_features=128, out_features=128, bias=True)
          (w_v): Linear(in_features=128, out_features=128, bias=True)
          (w_o): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): FeedForward(
          (linear1): Linear(in_features=128, out_features=128, bias=True)
          (linear2): Linear(in_features=128, out_features=128, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (relu): ReLU()
          (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): La

In [12]:
with torch.no_grad():
    output = model(
        src,
        tgt,
        cross_padding_mask = cross_padding_mask,
        tgt_is_causal = True,
        memory_is_causal = True
    )

print(output.shape)

torch.Size([3, 8, 14641])


In [13]:
start_seq = torch.tensor([[1],[1],[1]])
cross_padding_mask = create_cross_attention_mask(start_seq, src)
print(cross_padding_mask)
with torch.no_grad():
    output = model(
        src,
        start_seq,
        cross_padding_mask = cross_padding_mask,
        tgt_is_causal = True,
        memory_is_causal = True
    )

print(torch.argmax(output))


tensor([[[[False, False, False, False,  True,  True,  True]]],


        [[[False, False, False, False, False,  True,  True]]],


        [[[False, False,  True,  True,  True,  True,  True]]]])
tensor(21605)


In [14]:
print("=== Test Forward Pass ===")
with torch.no_grad():
    output = model(
        src,
        tgt,
        cross_padding_mask = cross_padding_mask,
        tgt_is_causal = True,
        memory_is_causal = True
    )
    print(f"Input source shape: {src.shape}")
    print(f"Input target shape: {tgt.shape}")
    print(f"Output shape: {output.shape}")  # [batch_size, tgt_len, vocab_size]

# Test generazione
print("\n=== Test Generazione ===")
with torch.no_grad():
    generated = model.generate(src, max_len=20, start_token=1, end_token=2)
    print(f"Generated sequence shape: {generated.shape}")
    print(f"First generated sequence: {generated[0].tolist()}")


=== Test Forward Pass ===
Input source shape: torch.Size([3, 7])
Input target shape: torch.Size([3, 8])
Output shape: torch.Size([3, 8, 14641])

=== Test Generazione ===
Generated sequence shape: torch.Size([3, 21])
First generated sequence: [1, 997, 997, 1488, 997, 1488, 997, 1488, 997, 1488, 997, 1488, 997, 1488, 997, 1488, 997, 1488, 997, 1488, 997]
